# 🧠 Agente RAG de Scouting Futbolístico con LangChain

Este notebook permite hacer consultas inteligentes sobre una base de datos de jugadores usando LangChain + FAISS + GPT-4.

In [33]:
# 📦 Instalar dependencias necesarias (si no están instaladas)
# Ejecutar solo una vez

!pip install langchain openai faiss-cpu python-dotenv pandas matplotlib



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
!pip install -U langchain-community


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
!pip install sentence-transformers


  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:


ERROR: Could not install packages due to an OSError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'c:\\users\\germán llorente\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\transformers\\models\\vit\\__pycache__\\modeling_vit.cpython-39.pyc.2541878068368'


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
!pip uninstall keras tensorflow -y


Found existing installation: keras 2.11.0
Uninstalling keras-2.11.0:
  Successfully uninstalled keras-2.11.0
Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Successfully uninstalled tensorflow-2.11.0


In [37]:
!pip install keras==2.11 tensorflow==2.11

  Using cached keras-2.11.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl.metadata (2.5 kB)
Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.11.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
!pip uninstall keras tensorflow transformers sentence-transformers -y
!pip install tensorflow==2.11 keras==2.11
!pip install transformers==4.30.2 
!pip install sentence-transformers==2.2.2
!pip install langchain faiss-cpu


Found existing installation: keras 2.11.0
Uninstalling keras-2.11.0:
  Successfully uninstalled keras-2.11.0
Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Successfully uninstalled tensorflow-2.11.0
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: sentence-transformers 2.2.2
Uninstalling sentence-transformers-2.2.2:
  Successfully uninstalled sentence-transformers-2.2.2
  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl.metadata (2.5 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)
Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.11.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sentence_transformers-2.2.2-py3-none-any.whl



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1️⃣ Cargar datos

In [42]:
import pandas as pd

# Cargar el dataset de jugadores
df_total = pd.read_csv("datos/datos_jugadores_v4/jugadores_total.csv")

# Filtrar jugadores con rating válido
df_total = df_total[df_total["rating"] != "S.V"].copy()
df_total = df_total.reset_index(drop=True)

# Mostrar algunos jugadores
df_total[['player_name', 'team', 'main_position', 'value_eur', 'rating']].head()


,player_name,team,main_position,value_eur,rating
0,Yuri Berchiche Izeta,Real Sociedad,Left Back,1800000.0,65.82662180990246
1,Danilo Luiz da Silva,Real Madrid,Right Back,16500000.0,68.35219356373398
2,Rene Krhin,Granada,Right Defensive Midfield,4500000.0,48.27438393818753
3,Adil Rami,Sevilla,Right Center Back,7500000.0,51.265068906982776
4,Abdoulaye Doucouré,Granada,Left Defensive Midfield,1700000.0,60.54364541416393


## 2️⃣ Preparar documentos para búsqueda semántica

In [47]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


# Selección de columnas clave para la búsqueda
campos_utiles = ["player_name", "team", "main_position", "value_eur", "rating"]
df_consulta = df_total[campos_utiles].copy()

# Crear columna 'text' combinando los datos relevantes para búsqueda semántica
df_consulta["text"] = df_consulta.apply(
    lambda row: f"{row['player_name']} juega en el {row['team']} como {row['main_position']}. "
                f"Valor de mercado: {row['value_eur'] / 1_000_000:.2f}M €. "
                f"Rating actual: {round(float(row['rating']), 2)}.",
    axis=1
)

# Ahora usamos el DataFrameLoader indicando explícitamente la columna 'text'
loader = DataFrameLoader(df_consulta, page_content_column="text")
docs = loader.load()

# División en fragmentos
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

# Crear índice FAISS con embeddings de OpenAI
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(split_docs, embedding)
retriever = vectorstore.as_retriever()


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

## 3️⃣ Crear cadena de consulta con GPT-4

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

# Cargar clave API desde archivo .env
load_dotenv()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#llm = ChatOpenAI(model_name="gpt-4", temperature=0)


# Cadena RAG
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)


## 4️⃣ Probar consulta

In [ ]:
# Cambia la consulta por lo que quieras preguntar
pregunta = "¿Qué jugadores valen más de 80 millones de euros?"
respuesta = qa_chain.run(pregunta)
print(respuesta)
